# Analyze All-Time Kicks

Every kick from every match is also saved in a separate branch of the database so that we can query analytics for players and their kicks across multiple matches. We call these the "all-time kicks" dataset.

We have stripped the player user names so that the dataset can be analyzed. This notebook summarizes the dataset.

In [1]:
import sys
sys.path.append("../")

from haxml.utils import (
    is_target_stadium,
    is_shot
)
import json
import pandas as pd

In [2]:
ALL_TIME_KICKS = "../data/all_time_kicks.json"
with open(ALL_TIME_KICKS, "r") as file:
    all_time_kicks = json.load(file)

In [3]:
df = pd.DataFrame(all_time_kicks.values())

## Dataset Summary

In [4]:
df.head().T

,0,1,2,3,4
fromName,Player 0,Player 1,Player 1,Player 1,Player 1
fromTeam,red,blue,blue,blue,blue
fromX,12,14,-219,-676,-651
fromY,0,0,11,-203,120
match,-MOTVkwbfE_IKa15MVn9,-MOTVkwbfE_IKa15MVn9,-MOTVkwbfE_IKa15MVn9,-MOTVkwbfE_IKa15MVn9,-MOTVkwbfE_IKa15MVn9
saved,1607903416292,1607903416293,1607903416294,1607903416296,1607903416297
scoreBlue,0,0,0,0,1
scoreLimit,2,2,2,2,2
scoreRed,1,1,1,1,1
stadium,NAFL Official Map v1,NAFL Official Map v1,NAFL Official Map v1,NAFL Official Map v1,NAFL Official Map v1


The three stadiums we will spend the most time playing in and analyzing are the two main NAFL (Futsal) maps (1v1, 2v2, 3v3, 4v4) and the Classic HaxBall map. We want to create a model that can be applied across multiple stadiums (as long as the stadium meets our assumptions), but we can focus our analysis and development on these three stadiums.

In [5]:
def summarize_df(df):
    n_matches = df["match"].nunique()
    n_players = df["fromName"].nunique()
    print(f"Rows: {df.shape[0]:,} kicks")
    print(f"Columns: {df.shape[1]:,} features")
    print(f"Matches: {n_matches:,} matches")
    print(f"Players: {n_players:,} unique players")

    
print("All-Time Kicks:")
summarize_df(df)
print()
print("From Target Stadiums:")
df_target = df[df["stadium"].apply(is_target_stadium)]
summarize_df(df_target)

All-Time Kicks:
Rows: 46,930 kicks
Columns: 22 features
Matches: 574 matches
Players: 317 unique players

From Target Stadiums:
Rows: 45,888 kicks
Columns: 22 features
Matches: 554 matches
Players: 309 unique players


### Table 1. Count of kicks, by stadium.

In [6]:
gp_stadium = df.groupby("stadium")["kick"].count()
pd.DataFrame(gp_stadium.sort_values(ascending=False)).reset_index()

,stadium,kick
0,NAFL Official Map v1,39307
1,NAFL 1v1/2v2 Map v1,4975
2,Classic,1606
3,Futsal 3x3 4x4 from HaxMaps,430
4,Futsal 1x1 2x2 from HaxMaps,116
5,Big,105
6,Small,102
7,Futsal AutoGK V1 By iShow from HaxMaps,99
8,Futsal 3v3 v2 from HaxMaps,79
9,Futsal 1v1 by Luchooo from HaxMaps,70


### Table 2. Count of matches, by stadium.

In [7]:
gp_matches = df.groupby("stadium")["match"].nunique()
pd.DataFrame(gp_matches.sort_values(ascending=False)).reset_index()

,stadium,match
0,NAFL Official Map v1,392
1,NAFL 1v1/2v2 Map v1,128
2,Classic,34
3,Futsal 3x3 4x4 from HaxMaps,4
4,Small,3
5,Futsal 1x1 2x2 from HaxMaps,3
6,Futsal AutoGK V1 By iShow from HaxMaps,2
7,Quidditch by Pael from HaxMaps,1
8,POWER 4v4 MNC from HaxMaps,1
9,Kafa Topu by Vhagar,1


### Table 3. Count of unique players who kicked (from), by stadium.

In [8]:
gp_players = df.groupby("stadium")["fromName"].nunique()
pd.DataFrame(gp_players.sort_values(ascending=False)).reset_index()

,stadium,fromName
0,NAFL Official Map v1,288
1,NAFL 1v1/2v2 Map v1,102
2,Classic,19
3,Futsal 3x3 4x4 from HaxMaps,12
4,Futsal 3v3 v2 from HaxMaps,8
5,Futsal 1x1 2x2 from HaxMaps,7
6,Big,7
7,Futsal AutoGK V1 By iShow from HaxMaps,4
8,Small,3
9,KERO CANZ Power Classic Dark v2 from HaxMaps,2


### Table 4. Count of kicks, by stadium and by type.

Showing target stadiums only.

In [9]:
df_types = df_target.groupby(["stadium", "type"])["kick"].count()
df_types_stadium = df_types.groupby("stadium", group_keys=False)
df_tsv = df_types_stadium.apply(lambda x: x.sort_values(ascending=False))
df_tsk = df_tsv.sort_index(level=0, key=lambda x: gp_stadium[x], ascending=False)
pd.DataFrame(df_tsk)

kick
stadium              type           
NAFL Official Map v1 steal     20242
                     save       5425
                     pass      12321
                     own_goal    260
                     goal        979
                     error        80
NAFL 1v1/2v2 Map v1  steal      2448
                     save       1597
                     pass        399
                     own_goal    124
                     goal        380
                     error        27
Classic              steal       938
                     save        509
                     pass         24
                     own_goal     39
                     goal         89
                     error         7

### Table 5. Fraction of kicks that are goals, by stadium.

In [10]:
"""
Calculate the fraction of kicks that are goals.
"""
def get_goal_fraction(kick_types):
    goals = sum(1 if val == "goal" else 0 for val in kick_types)
    return goals / len(kick_types)


gp_goals = df.groupby("stadium")["type"].agg(get_goal_fraction)
gp_goals_sorted = gp_goals.sort_index(key=lambda x: gp_stadium[x], ascending=False)
pd.DataFrame(gp_goals_sorted).reset_index()

,stadium,type
0,NAFL Official Map v1,0.024907
1,NAFL 1v1/2v2 Map v1,0.076382
2,Classic,0.055417
3,Futsal 3x3 4x4 from HaxMaps,0.025581
4,Futsal 1x1 2x2 from HaxMaps,0.051724
5,Big,0.028571
6,Small,0.049020
7,Futsal AutoGK V1 By iShow from HaxMaps,0.060606
8,Futsal 3v3 v2 from HaxMaps,0.063291
9,Futsal 1v1 by Luchooo from HaxMaps,0.057143
